## We’ll build a Prophet model to forecast global or country-specific temperatures for the next few years.

In [44]:
!pip install prophet

Defaulting to user installation because normal site-packages is not writeable


In [1]:
from prophet import Prophet
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error

### Load the data

In [2]:
df = pd.read_csv('../data/cleaned-GlobalLandTemperatures.csv')

### Convert the dt column to datetime

In [3]:
df['dt'] = pd.to_datetime(df['dt'])

### Split the data into train and test

In [8]:
train_size = int(len(df) * 0.8)
train = df[:train_size]
test = df[train_size:]

train

,dt,AverageTemperature,AverageTemperatureUncertainty,Country
0,1743-11-01,4.384,2.294,Åland
1,1744-04-01,1.530,4.680,Åland
2,1744-05-01,6.702,1.789,Åland
3,1744-06-01,11.609,1.577,Åland
4,1744-07-01,15.342,1.410,Åland
...,...,...,...,...
435843,1874-01-01,-3.265,1.736,Serbia
435844,1874-02-01,-1.752,0.676,Serbia
435845,1874-03-01,1.757,1.281,Serbia
435846,1874-04-01,10.931,0.563,Serbia


### Prepare the data for Prophet (prophet expects columns named 'ds', 'y')

In [5]:
train_prophet = train.rename(columns = {'dt': 'ds', 'AverageTemperature': 'y'})
test_prophet = test.rename(columns = {'dt': 'ds', "AverageTemperature": 'y'})

### Fit the model on the training data

In [6]:
model_prophet = Prophet(yearly_seasonality = True)
model_prophet.fit(train_prophet)

13:54:24 - cmdstanpy - INFO - Chain [1] start processing
13:57:37 - cmdstanpy - INFO - Chain [1] done processing


### Make the predictions for the same test period

In [7]:
future_prophet = model_prophet.make_future_dataframe(periods = len(test), freq = 'M')
forecast_prophet = model_prophet.predict(future_prophet)

ValueError: year 11093 is out of range

### Extract the forecast for the test set

In [ ]:
forecast_values_prophet = forecast_prophet['yhat'][-len(test):]

### Calculate evaluation metrics

In [ ]:
mae = mean_absolute_error(test_prophet['y'], forecast_values_prophet)
rmse = np.sqrt(mean_squared_error(test_prophet['y'], forecast_values_prophet))
mape = np.mean(np.abs((test_prophet['y'] - forecast_values_prophet) / test_prophet['y'])) * 100